# Implement natural gas supply scenarios in EU27 in the Ecoinvent database

This notebook creates a **new version of the Ecoinvent database (v3.9.1, cut-off system model)** that includes **activities for the production of natural gas-based energy in EU27**.

Modifications in the background database:

    - Creation of new activities for the production of natural-gas based energy in EU27 (activities basic meta-data provided in 'natural_gas_scenario_activities.yaml')
    - Creation of a new natural gas market group for EU27 that is re-linked to all the country-specific 'market for natural gas, high pressure'

In [ ]:
from pathlib import Path
import brightway2 as bw
import wurst
import yaml

## Basic setup

In [ ]:
# To be changed accordingly
BW_PROJECT = "NG_2022crisis_LCA"
ECOINVENT_DB = "ecoinvent 3.9.1 cutoff"
NEW_DB = "ecoinvent 3.9.1 cutoff, natural gas scenario EU27"

ECOINVENT_DIR = "./data/ecoinvent_3.9.1_cutoff/datasets"  # <- insert the path to ecoinvent 3.9 or 3.9.1 here (in case you don't have it in your brightway2 project)

DO_DATABASE = True

In [ ]:
# Access BW project
bw.projects.set_current(BW_PROJECT)

# biosphere3 setup
if "biosphere3" in bw.databases:
    print("Biosphere database already installed")
else:
    bw.bw2setup()

if ECOINVENT_DB in bw.databases:
    print("Ecoinvent database already installed")
else:
    ei391 = bw.SingleOutputEcospold2Importer(ECOINVENT_DIR, ECOINVENT_DB)
    ei391.apply_strategies()
    ei391.statistics()
    ei391.write_database()

# Import Ecoinvent database into wurst format
try:
    len(ei_db)
except NameError:
    ei_db = wurst.extract_brightway2_databases(ECOINVENT_DB)

In [ ]:
# Geographical scope
LOCATION = 'EU27'
COUNTRIES = [
             'BE', 'BG', 'CZ', 'DK', 'DE', 'EE', 'IE', 'GR', 'ES', 'FR', 'HR', 'IT', 'CY', 'LV', 'LT', 'LU', 'HU', 'MT', 'NL', 'AT', 'PL', 'PT', 'RO', 'SI', 'SK', 'FI', 'SE', # EU27 
             'RoE' # Rest of Europe - used in markets for natural gas in ecoinvent 3.9; may include non-EU27 countries (here assumed as other EU27 countries)
            ]

# Define some filters:
production = lambda x: x["type"] == "production"
technosphere = lambda x: x["type"] == "technosphere"
biosphere = lambda x: x["type"] == "biosphere"
eu27countries = lambda x: x["location"] in COUNTRIES

## Create scenario-specific activities

In [ ]:
# Some basic functions to modify activities

def add_product_to_exchanges(ds):
  '''
  Update exchanges list with an entry for the production type
  '''
  ds['exchanges'].append({
                          'name': ds['name'],
                          'product': ds['reference product'], 
                          'location': ds['location'],
                          'unit': ds['unit'],
                          'type': 'production',
                          'amount': 1,
                          'production volume': 0,
                          'database': ds['database']
                        })
  
def add_technosphere_exchange(ds, exc, amount):
  '''
  Update exchanges list with a new technosphere exchange
  '''
  ds['exchanges'].append({
                          'name': exc['name'],
                          'product': exc['reference product'], 
                          'location': exc['location'],
                          'unit': exc['unit'],
                          'type': 'technosphere',
                          'amount': amount,
                          'production volume': 0,
                          'database': exc['database']
                        })

def update_tehnosphere_exchange(exc, new_exc):
  '''
  Update an existing technosphere exchange
  '''  
  exc.update({
              'name': new_exc['name'],
              'product': new_exc['reference product'], 
              'location': new_exc['location'],
              'unit': new_exc['unit'],
              'database': new_exc['database']
            })

def add_biosphere_exchange(ds, exc, amount):
  '''
  Update exchanges list with a new biosphere exchange
  '''
  ef_code = [ef['code'] for ef in bw.Database('biosphere3') if ef['name'] == exc['name'] and 
                                                               ef['unit'] == exc['unit'] and 
                                                               ef['categories'] == exc['categories']][0]
  ds['exchanges'].append({
                          'name': exc['name'],
                          'categories': exc['categories'],
                          'unit': exc['unit'],
                          'type': 'biosphere',
                          'amount': amount,
                          'production volume': 0,
                          'database': 'biosphere3',
                          'input': ('biosphere3', ef_code)
                        })
    

def add_copy_to_new_location(ds, loc, db):
  '''
  Copy activity and change location; add copy to database and return the new activity
  '''
  ds_copy = wurst.transformations.geo.copy_to_new_location(ds, loc)
  db.append(ds_copy)
  return ds_copy

In [ ]:
# Import scenario-specific activities
scenario_activities = yaml.safe_load(
    Path("./data/natural_gas_scenario_activities.yaml").read_text()
)

# Add common fields: location, database, unique code, exchanges
for ds in scenario_activities:
    ds.update(
        {
            "location": LOCATION,
            "database": NEW_DB,
            "code": wurst.filesystem.get_uuid(),
            "exchanges": [],
        }
    )

# Link internally the new activities if inputs were provided in the .yaml file
for ds in scenario_activities:

    # Add product to exchanges
    add_product_to_exchanges(ds)

    # Default inputs provided:
    if "default_inputs" in ds:
        for exc in ds["default_inputs"]:
            exc_info = [
                i for i in scenario_activities if i["name"] == list(exc.keys())[0]
            ][0]
            add_technosphere_exchange(ds, exc_info, list(exc.values())[0])
        ds.pop("default_inputs")

## Link with the background Ecoinvent database

In [ ]:
 # Change the 'database' field in the ecoinvent database dictionary
for ds in ei_db:
    ds.update({'database': NEW_DB})
    for exc in ds['exchanges']:
        if exc['type'] == 'production' or exc['type'] == 'technosphere':
            if 'database' in exc:
                exc.update({'database': NEW_DB})

In [ ]:
# Extract scenario-specific activities that will be modified

# ... electricity from natural gas
elec_from_ng_EU27 = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity from NG'))
elec_from_NG = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity production, NG'))
elec_from_NG_chp = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity production, NG, CHP'))
elec_from_coal = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity production, coal'))
elec_from_coal_chp = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity production, coal, CHP'))
elec_from_nuclear = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity production, nuclear'))
elec_from_wind = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity production, wind'))
elec_from_pv = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity production, photovoltaic'))    
elec_from_biomass_chp = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity production, biomass, CHP'))
elec_from_oil = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity production, oil'))
elec_from_oil_chp = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity production, oil, CHP'))
elec_from_lignite = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity production, lignite'))
elec_from_lignite_chp = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity production, lignite, CHP'))
elec_from_hydro = wurst.get_one(scenario_activities, wurst.equals("name", 'electricity production, hydro'))

# ... heat from CHP
heat_from_chp_ng = wurst.get_one(scenario_activities, wurst.equals("name", 'heat production from NG, CHP'))
heat_from_chp_coal = wurst.get_one(scenario_activities, wurst.equals("name", 'heat production from coal, CHP'))
heat_from_chp_biomass = wurst.get_one(scenario_activities, wurst.equals("name", 'heat production from biomass, CHP'))
heat_from_chp_oil = wurst.get_one(scenario_activities, wurst.equals("name", 'heat production from oil, CHP'))
heat_from_chp_lignite = wurst.get_one(scenario_activities, wurst.equals("name", 'heat production from lignite, CHP'))

# ... heat in industry
heat_industry = wurst.get_one(scenario_activities, wurst.equals("name", 'heat from NG, industry'))
heat_industry_NG = wurst.get_one(scenario_activities, wurst.equals("name", 'heat production from NG, industry'))
heat_industry_coal = wurst.get_one(scenario_activities, wurst.equals("name", 'heat production from coal, industry'))
heat_industry_oil = wurst.get_one(scenario_activities, wurst.equals("name", 'heat production from oil, industry'))

# ... heat in households
heat_households_ng = wurst.get_one(scenario_activities, wurst.equals("name", 'heat production from NG, central or small-scale'))
heat_households_elec = wurst.get_one(scenario_activities, wurst.equals("name", 'heat production from electricity'))

# ... natural gas market
ng_market_EU27 = wurst.get_one(scenario_activities, wurst.equals("name", 'market group for natural gas, high pressure'))
ng_gas_distribution_EU27 = wurst.get_one(scenario_activities, wurst.equals("name", 'natural gas distribution within EU27, high pressure'))
ng_domestic_NL = wurst.get_one(scenario_activities, wurst.equals("name", 'natural gas, high pressure, domestic production in NL'))
ng_domestic_DE = wurst.get_one(scenario_activities, wurst.equals("name", 'natural gas, high pressure, domestic production in DE'))
ng_domestic_RO = wurst.get_one(scenario_activities, wurst.equals("name", 'natural gas, high pressure, domestic production in RO'))
ng_from_NO = wurst.get_one(scenario_activities, wurst.equals("name", 'natural gas, high pressure, import from NO'))
ng_from_RU = wurst.get_one(scenario_activities, wurst.equals("name", 'natural gas, high pressure, import from RU'))
ng_from_DZ = wurst.get_one(scenario_activities, wurst.equals("name", 'natural gas, high pressure, import from DZ'))
ng_from_AZ = wurst.get_one(scenario_activities, wurst.equals("name", 'natural gas, high pressure, import from AZ'))
lng_from_US = wurst.get_one(scenario_activities, wurst.equals("name", 'regasification LNG from US'))
lng_from_QA = wurst.get_one(scenario_activities, wurst.equals("name", 'regasification LNG from QA'))
lng_from_RU = wurst.get_one(scenario_activities, wurst.equals("name", 'regasification LNG from RU'))
lng_from_NG = wurst.get_one(scenario_activities, wurst.equals("name", 'regasification LNG from NG'))
lng_from_DZ = wurst.get_one(scenario_activities, wurst.equals("name", 'regasification LNG from DZ'))

# Extract ecoinvent activities that will be used for data collection
# ... for electricity
elec_market_Europe_high_voltage = wurst.get_one(ei_db, wurst.contains("name", "market group for electricity, high voltage"), wurst.contains("location", "Europe without Switzerland"))
elec_market_Europe_low_voltage = wurst.get_one(ei_db, wurst.contains("name", "market group for electricity, low voltage"), wurst.contains("location", "Europe without Switzerland"))    
# ... for heat CHP shares
heat_market_Europe_industrial_ng = wurst.get_one(ei_db, wurst.contains("name", "market for heat, district or industrial, natural gas"), wurst.contains("location", "Europe without Switzerland"))
heat_market_Europe_industrial_other_ng = wurst.get_one(ei_db, wurst.contains("name", "market for heat, district or industrial, other than natural gas"), wurst.contains("location", "Europe without Switzerland"))
# ... for heat in industry
heat_industry_Europe_ng = wurst.get_one(ei_db, wurst.contains("name", "heat production, natural gas, at industrial furnace >100kW"), wurst.contains("location", "Europe without Switzerland"))
heat_industry_Europe_coal = wurst.get_one(ei_db, wurst.contains("name", "heat production, at hard coal industrial furnace 1-10MW"), wurst.contains("location", "Europe without Switzerland"))
heat_industry_Europe_hfo = wurst.get_one(ei_db, wurst.contains("name", "heat production, heavy fuel oil, at industrial furnace 1MW"), wurst.contains("location", "Europe without Switzerland"))
# ... for heat in households
heat_smallscale_Europe_ng = wurst.get_one(ei_db, wurst.contains("name", "market for heat, central or small-scale, natural gas"), wurst.contains("location", "Europe without Switzerland"))
heat_pump_10kW_Europe = wurst.get_one(ei_db, wurst.contains("name", "heat production, air-water heat pump 10kW"), wurst.contains("location", "Europe without Switzerland"))
# ... for natural gas market
ng_market_Europe = wurst.get_one(ei_db, wurst.contains("name", "market group for natural gas, high pressure"), wurst.contains("location", "Europe without Switzerland"))
ng_domestic_NL_mix = wurst.get_one(ei_db, wurst.equals("name", 'natural gas, high pressure, domestic supply with seasonal storage'), wurst.equals("location", 'NL'))
ng_domestic_DE_mix = wurst.get_one(ei_db, wurst.equals("name", 'natural gas, high pressure, domestic supply with seasonal storage'), wurst.equals("location", 'DE'))
ng_domestic_RO_onshore = wurst.get_one(ei_db, wurst.equals("name", 'petroleum and gas production, onshore'), wurst.equals("reference product", 'natural gas, high pressure'), wurst.equals("location", 'RO'))    

### Electricity production from natural gas (and substitutes) in EU27

In [ ]:
# Contribution to the European market group for electricity per power generation technology and country
elec_market_EU27_tech_share = dict()

# Most power generation technologies are in the high voltage market, but solar PV is in the low voltage market

# Start with power generation technologies in the high voltage market...
for country in filter(technosphere, filter(eu27countries, elec_market_Europe_high_voltage["exchanges"])):
    country_share_in_Europe = country['amount']
    elec_market_country_high_voltage = wurst.get_one(ei_db, wurst.equals("name", country['name']), 
                                                        wurst.equals("reference product", country['product']),
                                                        wurst.equals("location", country['location']),
                                                    )
    for tech in filter(technosphere, elec_market_country_high_voltage["exchanges"]):
        if tech['product'] == 'electricity, high voltage' and tech['name'] != country['name'] and 'import from' not in tech['name']: # imports-exports are not accounted for...
            tech_info = (tech['name'], tech['product'], tech['location'])
            tech_share_in_Europe = country_share_in_Europe * tech['amount']
            if tech_info not in elec_market_EU27_tech_share:
                elec_market_EU27_tech_share.update({tech_info: tech_share_in_Europe})
            else:
                elec_market_EU27_tech_share[tech_info] += tech_share_in_Europe

# ... and do the same with solar PV in low voltage market    
for country in filter(technosphere, filter(eu27countries, elec_market_Europe_low_voltage["exchanges"])):
    country_share_in_Europe = country['amount']
    elec_market_country_low_voltage = wurst.get_one(ei_db, wurst.equals("name", country['name']), 
                                                    wurst.equals("reference product", country['product']),
                                                    wurst.equals("location", country['location']),
                                                    )            
    for tech in filter(technosphere, elec_market_country_low_voltage["exchanges"]):
        if 'electricity production, photovoltaic' in tech['name']:
            tech_info = (tech['name'], tech['product'], tech['location'])
            tech_share_in_Europe = country_share_in_Europe * tech['amount']
            if tech_info not in elec_market_EU27_tech_share:
                elec_market_EU27_tech_share.update({tech_info: tech_share_in_Europe})
            else:
                elec_market_EU27_tech_share[tech_info] += tech_share_in_Europe

# Populate scenario-specific electricity production activities in EU27 with country-weighted power generation technologies

def mix_of_countries_for_tech(scenario_ds, tech, full_list_techs, ei_db):
    '''
    This function computes the contribution of each country to the EU27 mix for a technology,
    finds the corresponding Ecoivent dataset, and add it to the scenario-speific inventory
    '''
    tech_by_country = {k:v for (k, v) in full_list_techs.items() if k[0] in tech}
    total_share = sum(tech_by_country.values())
    for i in tech_by_country:
        tech_amount = tech_by_country[i] / total_share
        tech_ds = wurst.get_one(ei_db, wurst.equals("name", i[0]), wurst.equals("reference product", i[1]), wurst.equals("location", i[2]))
        add_technosphere_exchange(scenario_ds, tech_ds, tech_amount)

mix_of_countries_for_tech(elec_from_NG, list(set([i[0] for i in elec_market_EU27_tech_share if 'electricity production, natural gas' in i[0]])), elec_market_EU27_tech_share, ei_db)
mix_of_countries_for_tech(elec_from_NG_chp, list(set([i[0] for i in elec_market_EU27_tech_share if 'heat and power co-generation, natural gas' in i[0]])), elec_market_EU27_tech_share, ei_db)
mix_of_countries_for_tech(elec_from_coal, list(set([i[0] for i in elec_market_EU27_tech_share if 'electricity production, hard coal' in i[0]])), elec_market_EU27_tech_share, ei_db)
mix_of_countries_for_tech(elec_from_coal_chp, list(set([i[0] for i in elec_market_EU27_tech_share if 'heat and power co-generation, hard coal' in i[0]])), elec_market_EU27_tech_share, ei_db)
mix_of_countries_for_tech(elec_from_nuclear, list(set([i[0] for i in elec_market_EU27_tech_share if 'electricity production, nuclear' in i[0]])), elec_market_EU27_tech_share, ei_db)
mix_of_countries_for_tech(elec_from_wind, list(set([i[0] for i in elec_market_EU27_tech_share if 'wind' in i[0]])), elec_market_EU27_tech_share, ei_db)
mix_of_countries_for_tech(elec_from_pv, list(set([i[0] for i in elec_market_EU27_tech_share if 'photovoltaic' in i[0]])), elec_market_EU27_tech_share, ei_db)                                               
# mix_of_countries_for_tech(elec_from_biomass, list(set([i[0] for i in elec_market_EU27_tech_share if 'heat and power co-generation, wood chips' in i[0]])), elec_market_EU27_tech_share, ei_db)
mix_of_countries_for_tech(elec_from_biomass_chp, list(set([i[0] for i in elec_market_EU27_tech_share if 'heat and power co-generation, wood chips' in i[0]])), elec_market_EU27_tech_share, ei_db)
mix_of_countries_for_tech(elec_from_oil, list(set([i[0] for i in elec_market_EU27_tech_share if 'electricity production, oil' in i[0]])), elec_market_EU27_tech_share, ei_db)
mix_of_countries_for_tech(elec_from_oil_chp, list(set([i[0] for i in elec_market_EU27_tech_share if 'heat and power co-generation, oil' in i[0]])), elec_market_EU27_tech_share, ei_db)
mix_of_countries_for_tech(elec_from_lignite, list(set([i[0] for i in elec_market_EU27_tech_share if 'electricity production, lignite' in i[0]])), elec_market_EU27_tech_share, ei_db)
mix_of_countries_for_tech(elec_from_lignite_chp, list(set([i[0] for i in elec_market_EU27_tech_share if 'heat and power co-generation, lignite' in i[0]])), elec_market_EU27_tech_share, ei_db)
mix_of_countries_for_tech(elec_from_hydro, list(set([i[0] for i in elec_market_EU27_tech_share if 'electricity production, hydro' in i[0]])), elec_market_EU27_tech_share, ei_db)

## Heat production from natural gas and coal CHP in EU27

In [ ]:
if DO_DATABASE:

    # Contribution to the European market for heat per CHP technology and country
    heat_market_EU27_chp_ng_tech_share = dict()
    heat_market_EU27_chp_coal_tech_share = dict()
    heat_market_EU27_chp_biomass_tech_share = dict()
    heat_market_EU27_chp_oil_tech_share = dict()
    heat_market_EU27_chp_lignite_tech_share = dict()
    
    # ... natural gas CHP
    for tech in filter(technosphere, filter(eu27countries, heat_market_Europe_industrial_ng["exchanges"])):
        tech_info = (tech['name'], tech['product'], tech['location'])
        if tech_info not in heat_market_EU27_chp_ng_tech_share:
            heat_market_EU27_chp_ng_tech_share.update({tech_info: tech['amount']})
        else:
            heat_market_EU27_chp_ng_tech_share[tech_info] += tech['amount']

    # ... coal CHP
    for tech in filter(technosphere, filter(eu27countries, heat_market_Europe_industrial_other_ng["exchanges"])):
        tech_info = (tech['name'], tech['product'], tech['location'])
        if tech_info not in heat_market_EU27_chp_coal_tech_share:
            heat_market_EU27_chp_coal_tech_share.update({tech_info: tech['amount']})
        else:
            heat_market_EU27_chp_coal_tech_share[tech_info] += tech['amount']    
            
    # ... biomass CHP
    for tech in filter(technosphere, filter(eu27countries, heat_market_Europe_industrial_other_ng["exchanges"])):
        tech_info = (tech['name'], tech['product'], tech['location'])
        if tech_info not in heat_market_EU27_chp_biomass_tech_share:
            heat_market_EU27_chp_biomass_tech_share.update({tech_info: tech['amount']})
        else:
            heat_market_EU27_chp_biomass_tech_share[tech_info] += tech['amount']    
            
    # ... oil CHP
    for tech in filter(technosphere, filter(eu27countries, heat_market_Europe_industrial_other_ng["exchanges"])):
        tech_info = (tech['name'], tech['product'], tech['location'])
        if tech_info not in heat_market_EU27_chp_oil_tech_share:
            heat_market_EU27_chp_oil_tech_share.update({tech_info: tech['amount']})
        else:
            heat_market_EU27_chp_oil_tech_share[tech_info] += tech['amount']    
            
    # ... lignite CHP
    for tech in filter(technosphere, filter(eu27countries, heat_market_Europe_industrial_other_ng["exchanges"])):
        tech_info = (tech['name'], tech['product'], tech['location'])
        if tech_info not in heat_market_EU27_chp_lignite_tech_share:
            heat_market_EU27_chp_lignite_tech_share.update({tech_info: tech['amount']})
        else:
            heat_market_EU27_chp_lignite_tech_share[tech_info] += tech['amount']    

    mix_of_countries_for_tech(heat_from_chp_ng, list(set([i[0] for i in elec_market_EU27_tech_share if 'heat and power co-generation, natural gas' in i[0]])), heat_market_EU27_chp_ng_tech_share, ei_db)
    mix_of_countries_for_tech(heat_from_chp_coal, list(set([i[0] for i in elec_market_EU27_tech_share if 'heat and power co-generation, hard coal' in i[0]])), heat_market_EU27_chp_coal_tech_share, ei_db)
    mix_of_countries_for_tech(heat_from_chp_biomass, list(set([i[0] for i in elec_market_EU27_tech_share if 'heat and power co-generation, wood chips' in i[0]])), heat_market_EU27_chp_biomass_tech_share, ei_db)
    mix_of_countries_for_tech(heat_from_chp_oil, list(set([i[0] for i in elec_market_EU27_tech_share if 'heat and power co-generation, oil' in i[0]])), heat_market_EU27_chp_oil_tech_share, ei_db)
    mix_of_countries_for_tech(heat_from_chp_lignite, list(set([i[0] for i in elec_market_EU27_tech_share if 'heat and power co-generation, lignite' in i[0]])), heat_market_EU27_chp_lignite_tech_share, ei_db)

## Heat production in industry in EU27

In [ ]:
if DO_DATABASE:
    # Add industrial heat production exchanges; by default, 1 MJ from natural gas and 0 from coal and HFO
    add_technosphere_exchange(heat_industry_NG, heat_industry_Europe_ng, 1)
    add_technosphere_exchange(heat_industry_coal, heat_industry_Europe_coal, 1)
    add_technosphere_exchange(heat_industry_oil, heat_industry_Europe_hfo, 1)

## Heat production in households in EU27

In [ ]:
if DO_DATABASE:
    # Heat production in households from natural gas
    for exc in filter(technosphere, heat_smallscale_Europe_ng["exchanges"]):
        exc_ds = wurst.get_one(ei_db, wurst.equals("name", exc['name']), 
                               wurst.equals("reference product", exc['product']),
                               wurst.equals("location", exc['location']),
                               )     
        add_technosphere_exchange(heat_households_ng, exc_ds, exc['amount'])

    # Heat production in households from electricity
    add_technosphere_exchange(heat_households_elec, heat_pump_10kW_Europe, 1)

    # Activity for energy savings is empty

## Natural gas market in EU27

In [ ]:
if DO_DATABASE: 

    # Domestic production

    # - NL and DE have both onshore and offshore production. The share of each can be found in the activity 'natural gas, high pressure, domestic supply with seasonal storage'
    for exc in filter(technosphere, ng_domestic_NL_mix["exchanges"]):
        if exc['product'] == 'natural gas, high pressure':
            exc_ds = wurst.get_one(ei_db, wurst.equals("name", exc['name']), 
                                   wurst.equals("reference product", exc['product']),
                                   wurst.equals("location", exc['location']),
                                  )
            add_technosphere_exchange(ng_domestic_NL, exc_ds, exc['amount'])

    for exc in filter(technosphere, ng_domestic_DE_mix["exchanges"]):
        if exc['product'] == 'natural gas, high pressure':
            exc_ds = wurst.get_one(ei_db, wurst.equals("name", exc['name']), 
                                   wurst.equals("reference product", exc['product']),
                                   wurst.equals("location", exc['location']),
                                  )
            add_technosphere_exchange(ng_domestic_DE, exc_ds, exc['amount'])    

    # - RO has only onshore production:
    add_technosphere_exchange(ng_domestic_RO, ng_domestic_RO_onshore, 1)

    # Imports and other exchanges; other inventory entires than imports and domestic production require that countries share sum up to 100%
    ng_market_EU27_imports = dict()
    EU27_countries_total_share = sum(country['amount'] for country in filter(technosphere, ng_market_Europe["exchanges"]) if country['location'] in COUNTRIES)
    ng_market_EU27_losses = 0
    ng_market_EU27_technosphere = dict()
    ng_market_EU27_biosphere = dict()

    for country in filter(technosphere, filter(eu27countries, ng_market_Europe["exchanges"])):
        country_share_in_Europe = country['amount']
        country_share_in_EU27 = country['amount'] / EU27_countries_total_share
        ng_market_country = wurst.get_one(ei_db, wurst.equals("name", country['name']), 
                                          wurst.equals("reference product", country['product']),
                                          wurst.equals("location", country['location']),
                                         )
        for exc in filter(technosphere, ng_market_country["exchanges"]):
            exc_info = (exc['name'], exc['product'], exc['location'])

            # Natural gas imports (add exchange share in Europe)
            if exc['product'] == 'natural gas, high pressure' and 'import from' in exc['name']:    
                exc_share_in_Europe = country_share_in_Europe * exc['amount']
                if exc_info not in ng_market_EU27_imports:
                    ng_market_EU27_imports.update({exc_info: exc_share_in_Europe})
                else:
                    ng_market_EU27_imports[exc_info] += exc_share_in_Europe

            # Market losses weighted for EU27 countries
            elif exc['name'] == country['name']:
                ng_market_EU27_losses += country_share_in_EU27 * exc['amount']

            # Utilities weighted for EU27 countries:
            else:
                exc_share_in_EU27 = country_share_in_EU27 * exc['amount']
                if exc_info not in ng_market_EU27_technosphere:
                    ng_market_EU27_technosphere.update({exc_info: exc_share_in_EU27})
                else:
                    ng_market_EU27_technosphere[exc_info] += exc_share_in_EU27

        # Biosphere emissions
        for exc in filter(biosphere, ng_market_country["exchanges"]):
            exc_info = (exc['name'], exc['categories'], exc['unit'])
            exc_share_in_EU27 = country_share_in_EU27 * exc['amount']                
            if exc_info not in ng_market_EU27_biosphere:
                ng_market_EU27_biosphere.update({exc_info: exc_share_in_EU27})
            else:
                ng_market_EU27_biosphere[exc_info] += exc_share_in_EU27
    
    # Populate the new market for natural gas in EU27...
                            
    # ... with imports by pipeline
    mix_of_countries_for_tech(ng_from_NO, ['natural gas, high pressure, import from NO'], ng_market_EU27_imports, ei_db)
    mix_of_countries_for_tech(ng_from_RU, ['natural gas, high pressure, import from RU'], ng_market_EU27_imports, ei_db)
    mix_of_countries_for_tech(ng_from_DZ, ['natural gas, high pressure, import from DZ'], ng_market_EU27_imports, ei_db)
    mix_of_countries_for_tech(ng_from_AZ, ['natural gas, high pressure, import from TR'], ng_market_EU27_imports, ei_db) # ! Assume that gas comes directly from Azerbaiyan

    # ... with LNG imports
    lng_from_US_ds = wurst.get_one(ei_db, wurst.equals("name", 'evaporation of natural gas, import from US'), 
                                          wurst.equals("reference product", 'natural gas, high pressure'),
                                          wurst.equals("location", 'Europe without Switzerland'))
    lng_from_QA_ds = wurst.get_one(ei_db, wurst.equals("name", 'evaporation of natural gas, import from QA'), 
                                          wurst.equals("reference product", 'natural gas, high pressure'),
                                          wurst.equals("location", 'Europe without Switzerland'))
    lng_from_RU_ds = wurst.get_one(ei_db, wurst.equals("name", 'evaporation of natural gas, import from RU'), 
                                          wurst.equals("reference product", 'natural gas, high pressure'),
                                          wurst.equals("location", 'Europe without Switzerland'))
    lng_from_NG_ds = wurst.get_one(ei_db, wurst.equals("name", 'evaporation of natural gas, import from NG'), 
                                          wurst.equals("reference product", 'natural gas, high pressure'),
                                          wurst.equals("location", 'Europe without Switzerland'))
    lng_from_DZ_ds = wurst.get_one(ei_db, wurst.equals("name", 'evaporation of natural gas, import from DZ'), 
                                          wurst.equals("reference product", 'natural gas, high pressure'),
                                          wurst.equals("location", 'Europe without Switzerland'))

    add_technosphere_exchange(lng_from_US, lng_from_US_ds, 1)
    add_technosphere_exchange(lng_from_QA, lng_from_QA_ds, 1)
    add_technosphere_exchange(lng_from_RU, lng_from_RU_ds, 1)
    add_technosphere_exchange(lng_from_NG, lng_from_NG_ds, 1)
    add_technosphere_exchange(lng_from_DZ, lng_from_DZ_ds, 1)

    # ... with market losses
    add_technosphere_exchange(ng_market_EU27, ng_market_EU27, ng_market_EU27_losses)

    # ... with utilities for gas distribution
    for exc in ng_market_EU27_technosphere:
        exc_ds = wurst.get_one(ei_db, wurst.equals("name", exc[0]), 
                                      wurst.equals("reference product", exc[1]),
                                      wurst.equals("location", exc[2])
                              )
        add_technosphere_exchange(ng_gas_distribution_EU27, exc_ds, ng_market_EU27_technosphere[exc])

    # ... with biosphere emissions
    for exc in ng_market_EU27_biosphere:
        add_biosphere_exchange(ng_market_EU27, {'name': exc[0], 'categories': exc[1], 'unit': exc[2]}, ng_market_EU27_biosphere[exc])


#### Integrate the new natural gas market for EU27 in the Ecoinvent database

In [ ]:
if DO_DATABASE:
    for ds in filter(eu27countries, ei_db):
        if ds['name'] == 'market for natural gas, high pressure':
            ds_product = [exc for exc in ds["exchanges"] if exc['type'] == 'production']
            ds["exchanges"] = ds_product
            add_technosphere_exchange(ds, ng_market_EU27, 1)

# Write database

In [ ]:
if DO_DATABASE:
    if NEW_DB in bw.databases:
        del bw.databases[NEW_DB]
    wurst.write_brightway2_database(ei_db + scenario_activities, NEW_DB)

## Test new database

In [ ]:
new_ei_db = bw.Database(NEW_DB)
random_func_unit = {new_ei_db.random(): 1}
lca = bw.LCA(random_func_unit)
lca.lci()
